## Training a specialised model (CNN+RNN) for Othello/Reversi

This notebook present a new approach to estimate the next move to play in a game of Othello using Supervised Learning. The datasets come from the [Fédération Française d'Othello](https://www.ffothello.org/informatique/la-base-wthor/). The model input is on one hand the board state which will feed in a Convolutional Neural Network (CNN) and on the other hand the history of the game which will feed in a Recurrent Neural Network (RNN). The output of the model is the next move to play.

### Data Handling

#### Extracting data from the WThor database